In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import xarray as xr
import numpy as np
import os
from glob import glob
from mpl_toolkits.basemap import Basemap
from numpy import meshgrid
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import matplotlib.colors as mcolors
import cartopy.feature as cfeature
import itertools
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter, LatitudeLocator
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, TwoSlopeNorm
import pandas as pd
import math
from scipy.stats import percentileofscore as pos
from datetime import datetime
import datetime as dt
from multiprocessing import Pool

import json

#Personal functions
from function import funs
from function import preprocessUtils as putils
from function import conf
from function import masks

/glade/work/klesinger/conda-envs/tf212gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Test with small subset of data
region_name="CONUS"
print(f'Loading data from base directory {conf.gefsv12_data}')
data_RZSM = xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/soilw_bgrnd/soilw_bgrnd_EMC_2012-09-26.nc')
data_z200 = xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/hgt_pres/hgt_pres_EMC_2016-04-20.nc')
data_pwat = xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/pwat_eatm/pwat_eatm_EMC_2014-04-30.nc')
data_tmax = xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/tmax_2m/tmax_2m_EMC_2016-04-20.nc')


mask = masks.load_mask(region_name)

save_dir = 'Outputs/images_for_diagram'
os.system(f'mkdir -p {save_dir}')

Loading data from base directory /glade/work/klesinger/FD_RZSM_deep_learning/Data/reforecast/GEFSv12


0

# The purpose of this next code block is to simply create some static images to be used in some figure diagrams for the UNET architecture

In [9]:

def plot_and_save(file, name, lead_index):
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    new_X_coords = putils.create_new_X_coord_values(file)
    file = file.assign_coords({'X':new_X_coords})
    file = putils.restrict_to_bounding_box(file,mask)
    file = file.isel(S=0).isel(L=lead_index,M=0)
    
    
    ax = plt.axes(projection=ccrs.PlateCarree())
    file[putils.xarray_varname(file)].plot.pcolormesh(ax=ax,cmap='bwr')
    ax.coastlines()
    plt.savefig(f'{save_dir}/{name}_{lead_index}.png')
    plt.close()


plot_and_save(data_z200, 'z200', 0)
plot_and_save(data_z200, 'z200', 34)
plot_and_save(data_pwat, 'pwat', 0)
plot_and_save(data_pwat, 'pwat', 34)
plot_and_save(data_tmax, 'tmax', 0)
plot_and_save(data_tmax, 'tmax', 34)

In [10]:
def plot_and_save_RZSM(file, name, lead_index, model):
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    new_X_coords = putils.create_new_X_coord_values(file)
    file = file.assign_coords({'X':new_X_coords})
    file = putils.restrict_to_bounding_box(file,mask)
    file = file.isel(S=0).isel(L=lead_index,M=model)
    file = xr.where(file[putils.xarray_varname(file)]>=0.4,np.nan,file)
    
    ax = plt.axes(projection=ccrs.PlateCarree())
    file[putils.xarray_varname(file)].plot.pcolormesh(ax=ax,cmap='bwr')
    ax.coastlines()
    plt.savefig(f'{save_dir}/{name}_{lead_index}_mod{model}.png')
    plt.close()



In [12]:
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2001-05-02.nc'), 'RZSM', 7,0)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2001-05-02.nc'), 'RZSM', 34,0)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2001-05-02.nc'), 'RZSM', 34,7)
# plot_and_save_RZSM(data_RZSM, 'RZSM', 13,10)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2012-09-05.nc'), 'RZSM', 9,0)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2012-09-05.nc'), 'RZSM', 33,0)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2012-09-05.nc'), 'RZSM', 33,2)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2012-09-05.nc'), 'RZSM', 33,9)

plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2010-10-13.nc'), 'RZSM', 7,0)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2010-10-13.nc'), 'RZSM', 32,8)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2010-10-13.nc'), 'RZSM', 32,6)

plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2019-12-25.nc'), 'RZSM', 10,0)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2019-12-25.nc'), 'RZSM', 31,0)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2019-12-25.nc'), 'RZSM', 31,2)
plot_and_save_RZSM(xr.open_dataset(f'{conf.gefsv12_data}/{region_name}/baseline_RZSM_anomaly/soilw_bgrnd_2019-12-25.nc'), 'RZSM', 31,9)

In [ ]:
file=data_RZSM

In [ ]:
file[putils.xarray_varname(file)]